In [85]:
import pandas as pd
import numpy as np
import warnings

from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [86]:
data = pd.read_csv(path+'data/ml_input.csv')
merged = pd.read_csv(path+'data/merged.csv')
parameters = pd.read_csv(path+'parameters/neutralnetworkregressor.csv')

In [87]:
N = 2022

In [88]:
params_to_drop = ['season', 'round', 'podium', 'driver_points_from']

In [89]:
df = data.copy()

train = df[df.season < N]

scaler = StandardScaler()

X_train = train.drop(params_to_drop, axis=1)
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

y_train = np.asarray(train.driver_points_from.values)

In [90]:
print(*data.columns)

season round podium q_delta starting_grid driver_points_before constructor_points_before driver_points_from driver_points_per points_percentage driver_last_3 constructor_last_3 stage_q1 stage_q2 stage_q3


In [91]:
def get_predictions(X_train, y_train, model):
    points = []
    winners = []
    for circuit in df[df.season == N]['round'].unique():

        test = df[(df.season == N) & (df['round'] == circuit)]
        X_test = test.drop(params_to_drop, axis=1)
        y_test = test.driver_points_from

        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

        # make predictions
        prediction_df = pd.DataFrame(model.predict(X_test), columns=['predicted_points'])
        prediction_df['actual_points'] = y_test.reset_index(drop=True)

        prediction_df['predicted_winner'] = prediction_df.predicted_points.map(lambda x: 1 if x == prediction_df.predicted_points.max() else 0)
        prediction_df['actual_winner'] = prediction_df.actual_points.map(lambda x: 1 if x == prediction_df.actual_points.max() else 0)

        winners += list(prediction_df.predicted_winner.values)
        points += list(prediction_df.predicted_points.values)
        
        # Retrain model
        # X_train = pd.concat([X_train, X_test])
        # y_train = np.append(y_train, y_test)


        # model.fit(X_train, y_train)

    return points, winners

In [92]:
params = parameters.iloc[0]
hidden_layer_sizes = eval(params.hidden_layer_sizes)
activation = params.activation
solver = params.solver
alpha = params.alpha

params

Unnamed: 0                          0
model                    nn_regressor
hidden_layer_sizes    (80, 20, 40, 5)
activation                   identity
solver                          lbfgs
alpha                          0.0001
score                        0.681818
Name: 0, dtype: object

In [93]:
test = df[(df.season == N)]

model = MLPRegressor(
    hidden_layer_sizes=hidden_layer_sizes,
    activation=activation, 
    solver=solver, 
    alpha=alpha, 
    random_state=1)
    
model.fit(X_train, y_train)

points, winner = get_predictions(X_train, y_train, model)

test['predicted_points'] = points
test['predicted_winner'] = winner

In [94]:
len(test.query('predicted_winner == 1 & podium == 1'))/ test['round'].nunique()

0.5263157894736842

In [95]:
df = test[['season', 'round', 'podium', 'predicted_winner','starting_grid','driver_points_from', 'predicted_points']]

In [96]:
df.query('predicted_winner == 1')

,season,round,podium,predicted_winner,starting_grid,driver_points_from,predicted_points
3111,2022,1,1,1,1,26.0,8.571416
3132,2022,2,2,1,2,19.0,15.540562
3149,2022,3,1,1,1,26.0,17.563685
3169,2022,5,2,1,1,18.0,18.736559
3206,2022,6,20,1,1,0.0,18.808257
3210,2022,7,4,1,1,12.0,17.205921
3245,2022,8,19,1,1,0.0,16.995191
3247,2022,9,1,1,1,25.0,18.440165
3273,2022,10,7,1,2,6.0,18.193961
3287,2022,12,1,1,2,25.0,17.838006


In [97]:
df.query('round == 10')

,season,round,podium,predicted_winner,starting_grid,driver_points_from,predicted_points
3267,2022,10,1,0,1,25.0,14.064025
3268,2022,10,2,0,4,18.0,13.969106
3269,2022,10,3,0,5,16.0,10.022572
3270,2022,10,4,0,3,12.0,14.356419
3271,2022,10,5,0,7,10.0,5.176380
3272,2022,10,6,0,6,8.0,7.145126
3273,2022,10,7,1,2,6.0,18.193961
3274,2022,10,8,0,19,4.0,-0.230183
3275,2022,10,9,0,18,2.0,-0.053542
3276,2022,10,10,0,17,1.0,0.145515
